In [2]:
import os
import cv2
import tensorflow as tf
from pathlib import Path
import numpy as np
from tensorflow.keras.utils import img_to_array
from tensorflow.keras import layers
from tensorflow.keras import Model
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout

In [2]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('bank_notes.h5')

In [3]:
w1 = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
w2 = widgets.Button(
    description='Predict',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='image' # (FontAwesome names without the `fa-` prefix)
)
w2.style.button_color = 'silver'

debug_view = widgets.Output()

def on_upload_change(change):

    main_container.children = main_container.children[:-1]
    
def predict(image):
    img=cv2.imdecode(np.frombuffer(image, np.uint8), -1)
    img=cv2.resize(img, (150,150), interpolation = cv2.INTER_AREA)  
    img=img/255
    predictions=new_model.predict(np.expand_dims(img, axis=0))
    pre_class=predictions.argmax()
    if predictions[0]>=0.6:
        print("This image contains $50")
    else:
        print("This image contains  $100")    

@debug_view.capture(clear_output=True)
def show_image(btn):
   
    if(w1.value):
    # if(len(list(w1.value.items()))>0):
        # image = list(w1.value.items())[0][1]['content']
        image = w1.value[0]["content"]
        w3=widgets.Image(
        value=image,
        format='png',
        width=100,
        height=200,
        )
        
        main_container.children = main_container.children  + (VBox([w3,debug_view]),)
        
        predict(image)

    else:
        print("Please upload an image")

w1.observe(on_upload_change, names='value')        
w2.on_click(show_image)

main_container = HBox([w1,w2,VBox([debug_view])])
main_container